In [1]:
from download_websites import download_websites
from generate_scraping_tasks import generate_scraping_tasks, get_body
from generate_solution_code import generate_solution_code

import pandas as pd

In [2]:
websites_csv = 'websites.csv'
save_folder = 'downloaded_pages'
NUM_TASKS_PER_WEBSITE = 2

# Download websites
# download_websites(websites_csv, save_folder)

# Generate scraping tasks
websites = pd.read_csv(websites_csv)
for _, row in websites.iterrows():
    category, website, page, link = row
    HTML_file = f'{save_folder}/{website}.html'
    HTML_body = get_body(HTML_file)[:1000]
    generate_scraping_tasks(link, website, category, HTML_body, NUM_TASKS_PER_WEBSITE)
    with open(f'scraping_tasks/{website}.txt', 'r') as f:
        scraping_tasks = pd.DataFrame([{'task': task} for task in f.readlines()])

    for i, task in scraping_tasks.iterrows():
        task = task['task']
        generate_solution_code(website, HTML_file, category, HTML_body, task, i)

In [5]:
import amazon_1

In [8]:
from lxml import etree
from io import StringIO

def extract_text_objects(html_content):
    # Parse the HTML using lxml
    parser = etree.HTMLParser()
    tree = etree.parse(StringIO(html_content), parser)
    
    # Get all elements that have text directly inside them
    elements_with_text = [elem for elem in tree.xpath('//*') if elem.text and not elem.getchildren()]

    # Exclude script and style tags
    elements_with_text = [elem for elem in elements_with_text if elem.tag not in ['script', 'style']]
    
    # Extract the HTML and xpath for each element
    htmls = [etree.tostring(elem, encoding='unicode') for elem in elements_with_text]
    xpaths = [tree.getpath(elem) for elem in elements_with_text]
    
    return htmls, xpaths

html = open('downloaded_pages/amazon.html', 'r').read()
htmls, xpaths = extract_text_objects(html)
print(htmls[300])
print(xpaths[30])
print(len(htmls))

# TODO: Make a function that generates a snippet of some of the HTML and XPaths.
# They should be selected equally for example or randomly.
# Also if some of them are too long then they should be truncated.
# Also remove all other HTML options from the objects except for class and id.
# If the text is too long it should be truncated.
# Include both objects with originally long text and short text.
# Include objects with as many different tags as possible, fx div, p, span, etc.

<span aria-hidden="true">$39.99</span>
/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[1]/div/div/select/option[23]
800
